# C - TIME SERIES SMOOTHING
### NO PRE-REQUIREMENT EXCEPT UPDATED SAR_INDEX_STATS TABLE
### Can be run in parallel with (B)
### This program carries out time-series gap filling and smoothing (gaussian filter) for a selection of indexes using the following function:
#### smooth_time_serie(dteDebut, dteFin, dteExe)
### For each SAR, this function operates the following steps:
### 1) Listing of sentinel 2 for which we want to obtain smoothed profiles
### 2) Selection of index raw data in the sar_index_stats tables and outlier removal for the period between dteDebut and dteFin
### 3) Gap filling and smoothing
### 4) Smoothed time series are saved in tables
#### There is one table per index per site
#### Tables are named as following: (id_segment)_(index)_(dteExe)_smoothed


In [ ]:
import os, sys, datetime
sys.path.append("/home/gswinnen/SARSAR_Package_RenPri/code/") # emplacement des modules RenPri

#sys.path.append("/home/issep/sarsar-issep/SARSAR_utils/")                   # emplacement des modules RenPri
#sys.path.append("/home/issep/sarsar-issep/SARSAR_utils/rme_chg_detection_module/") # emplacement de la fonction de Mattia

from issep import sarsar_admin
from os.path import join
from lecture_ini import config

In [ ]:
def smooth_time_serie(dteDebut, dteFin, dteExe):
    
    # List all indices
    indices = ['BAI','BI','BI2','NDVI','SBI']
        
    # Define Database connection parameters
    # NOTE: password is in ~/.pgpass
    credentials = config(section='postgresql')

    db_credentials = {
        'host': credentials['host'],
        'user': credentials['user'],
        'db' : credentials['database']
    }

    # ALWAYS prepare env et the beginning
    print('> Preparing env (DB credentials, etc)')
    sarsar_admin.prepare_env(db_credentials)
    
    # Process all sar_id_segments
    conn = sarsar_admin._create_or_get_db_connection()
    cur = None

    try:
        import psycopg2.extras
        cur = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)
        cur.execute('SELECT DISTINCT sar_id_segment FROM sar_index_stats;')    
        result = cur.fetchall()
        for row in result:
            sar_id_segment = dict(row)
            
            
            for indice in indices :
                print('> Processing smoothing for %s, %s' % (sar_id_segment, indice))
                raw_profile = sarsar_admin.fetch_sar_index_stats_by_sar_id(sar_id_segment['sar_id_segment'], indice, dteDebut, dteFin )
                
                # Supprime les outliers du dataframe
                no_outlier_profile = sarsar_admin.remove_outliers_from_stats(raw_profile)

                # Lissage
                smoothed_profile = sarsar_admin.compute_smoothed_time_series_stats(no_outlier_profile)

                # transpose les deux arrays dans un dictionnaire tel qu'attendu pour la boucle suivante...
                smoothed_dates = smoothed_profile[0]
                smoothed_values = smoothed_profile[1]
                smoothed_profile_dico = {}

                for i in range(len(smoothed_dates)):
                    smoothed_profile_dico[smoothed_dates[i].strftime("%Y-%m-%d")] = smoothed_values[i]

                # (re)crée LA table "{sar_id_segment}_{indice}_{dteExe}_smoothed" qui doit accueillir ces données
                table_name = '{0}_{1}_{2}_smoothed'.format(sar_id_segment['sar_id_segment'], indice, dteExe)
                strSQL = 'DROP TABLE IF EXISTS "{0}";'.format(table_name)  # Encadrer le nom de table pour tolérer les chiffres en début et les tirets au milieu
                cur.execute(strSQL)
                conn.commit()

                strSQL = 'CREATE TABLE IF NOT EXISTS "{0}" (dte DATE, indice NUMERIC);'.format(table_name)  #, ', '.join(type_valeurs))
                cur.execute(strSQL)

                for item in smoothed_profile_dico:
                    strSQL = 'INSERT INTO "{0}" (dte, indice) VALUES (\'{1}\', {2});'.format(table_name, item, smoothed_profile_dico[item])  # , (', '.join(champs)), str(tuple(valeurs)))
                    cur.execute(strSQL)

            # Poste toutes les modifications
            conn.commit()
            
        cur.close()
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
            
    # ALWAYS release env at the end
    print('> Releasing env')
    sarsar_admin.release_env()